<a href="https://colab.research.google.com/github/sinhvienfpt/UsedCarPricePrediction/blob/develop/OldCarPrice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
url = "https://raw.githubusercontent.com/sinhvienfpt/UsedCarPricePrediction/main/assets/data/cleaned.csv"

In [8]:
import pandas as pd
pd.options.mode.chained_assignment = None #Ignore pandas warning

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Grid CV
from sklearn.model_selection import GridSearchCV

In [9]:
import warnings
warnings.filterwarnings('ignore')

# Modeling

In [10]:
df = pd.read_csv(url)
df.head()

,Name,Location,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,Price,Brand,...,Mitsubishi,Nissan,Porsche,Renault,Skoda,Smart,Tata,Toyota,Volkswagen,Volvo
0,Maruti Wagon R LXI CNG,Mumbai,72000,3,21.01,998.0,58.16,5.0,1.75,Maruti,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Hyundai Creta 1.6 CRDi SX Option,Pune,41000,3,19.67,1582.0,126.20,5.0,12.50,Hyundai,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Honda Jazz V,Chennai,46000,3,18.20,1199.0,88.70,5.0,4.50,Honda,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Maruti Ertiga VDI,Chennai,87000,3,20.77,1248.0,88.76,7.0,6.00,Maruti,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,40670,2,15.20,1968.0,140.80,5.0,17.74,Audi,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Data spliting

In [11]:
df.dropna(inplace=True)

In [12]:
X,y = df.drop(['Name', 'Location', 'Brand','Price'],axis=1),df["Price"]

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

## Evaluating regression models (with default parameters)

Test run with **Linear Regression**

In [13]:
# Linear Regression
lr = LinearRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
print("Linear Regression")
print("MSE : ",mean_squared_error(y_test,y_pred))
print("MAE : ",mean_absolute_error(y_test,y_pred))
print("R2 : ",r2_score(y_test,y_pred))
print()

Linear Regression
MSE :  2.868469543200723
MAE :  1.2037084309151775
R2 :  0.8174260639771891



In [14]:
models = {
    'Linear Regression' : LinearRegression(),
    'Random Forest' : RandomForestRegressor(),
    'Decision Tree' : DecisionTreeRegressor(),
    'KNN' : KNeighborsRegressor(),
    'Gradient Boosting' : GradientBoostingRegressor(),
    'XGBoost' : XGBRegressor(),
    'LightGBM' : LGBMRegressor(force_row_wise=True)
}

result = {
    'Model' : [],
    'MSE' : [],
    'MAE' : [],
    'R2' : []
}

In [15]:
for model_name,model in models.items():
  model.fit(X_train,y_train)
  y_pred = model.predict(X_test)
  result['Model'].append(model_name)
  result['MSE'].append(mean_squared_error(y_test,y_pred))
  result['MAE'].append(mean_absolute_error(y_test,y_pred))
  result['R2'].append(r2_score(y_test,y_pred))


[LightGBM] [Info] Total Bins 882
[LightGBM] [Info] Number of data points in the train set: 3953, number of used features: 38
[LightGBM] [Info] Start training from score 6.154430


In [16]:
result_df = pd.DataFrame(result)
result_df.sort_values(by='R2',ascending=False)

,Model,MSE,MAE,R2
5,XGBoost,1.607149,0.774832,0.897707
6,LightGBM,1.745633,0.808100,0.888893
1,Random Forest,1.881803,0.862756,0.880226
4,Gradient Boosting,2.032270,0.943706,0.870649
0,Linear Regression,2.868470,1.203708,0.817426
2,Decision Tree,3.549875,1.118278,0.774056
3,KNN,9.948519,2.291432,0.366791


## GridCV

In [17]:
# Define of models and corresponding parameter spaces
models = {
    'Linear Regression': (LinearRegression(), {}),
    'Random Forest': (RandomForestRegressor(), {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10]
    }),
    'Decision Tree': (DecisionTreeRegressor(), {
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10]
    }),
    'KNN': (KNeighborsRegressor(), {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance']
    }),
    'Gradient Boosting': (GradientBoostingRegressor(), {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1, 0.3],
        'max_depth': [3, 5, 7]
    }),
    'XGBoost': (XGBRegressor(), {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1, 0.3],
        'max_depth': [3, 5, 7]
    }),
    'LightGBM': (LGBMRegressor(force_row_wise=True), {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1, 0.3],
        'max_depth': [3, 5, 7]
    })
}

In [18]:
result = {
    'Model': [],
    'Best Parameters': [],
    'MSE': [],
    'MAE': [],
    'R2': []
}

In [19]:
for model_name, (model, params) in models.items():
    grid_search = GridSearchCV(model, params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)

    result['Model'].append(model_name)
    result['Best Parameters'].append(grid_search.best_params_)
    result['MSE'].append(mean_squared_error(y_test, y_pred))
    result['MAE'].append(mean_absolute_error(y_test, y_pred))
    result['R2'].append(r2_score(y_test, y_pred))

result_df = pd.DataFrame(result)
print(result_df)

[LightGBM] [Info] Total Bins 882
[LightGBM] [Info] Number of data points in the train set: 3953, number of used features: 38
[LightGBM] [Info] Start training from score 6.154430
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

In [20]:
result_df.sort_values(by='R2',ascending=False,inplace=True)
result_df.to_csv('Models_with_Best_Param.csv',index=False)

In [21]:
result_df

,Model,Best Parameters,MSE,MAE,R2
5,XGBoost,"{'learning_rate': 0.3, 'max_depth': 3, 'n_esti...",1.506640,0.754931,0.904105
6,LightGBM,"{'learning_rate': 0.3, 'max_depth': 3, 'n_esti...",1.612402,0.783649,0.897373
4,Gradient Boosting,"{'learning_rate': 0.3, 'max_depth': 3, 'n_esti...",1.695730,0.774221,0.892069
1,Random Forest,"{'max_depth': 30, 'min_samples_split': 2, 'n_e...",1.874319,0.853213,0.880702
0,Linear Regression,{},2.868470,1.203708,0.817426
2,Decision Tree,"{'max_depth': 10, 'min_samples_split': 10}",2.971040,1.103271,0.810898
3,KNN,"{'n_neighbors': 9, 'weights': 'distance'}",9.071643,2.163474,0.422603


## Conclusion
As we can see, XGBoost bring the highest R2 (~0.896) with the its best parameters {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300}.

I'll use it for the last model.

In [22]:
# Training with the full dataset
xgb = XGBRegressor(learning_rate=0.1,max_depth=5,n_estimators=300)
xgb.fit(X,y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=300, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [23]:
# Save the model
import pickle
pickle.dump(xgb,open('model.pkl','wb'))